## 1. Getting started: Kaggle's Titanic Competition

Kaggle is already established as the best place which hosts machine learning competitions. If you do not know it already, then it's time to do it.

__[Titanic Competition](https://www.kaggle.com/c/titanic)__ is perhaps the first competition which one should try. Of course, if you are already an experienced data scientist, than you can skip to an advanced competition.

The purpose of the competition is to learn if a passenger has survived or not. We illustrate some steps and ideas one can apply to compete in this learning competition using the available tools one can find in rapaio library.

In [1]:
%load rapaio-bootstrap

### 1.2 Get the data

The purpose of the competition is to predict which passengers have survived or not. The available data has two parts. The first part consists in a data set which contains what happened with some passengers and some related information like sex, cabin, age, class, etc. This data set contains information regarding their survival. The purpose why this data set contains survival data is because it will be used to train a model which learns how to decide if a passenger survives or not. This is the `train.csv`. The other file is a data set which contains data about another set of passenger, this time without knowing if they survived or not. They contain, however an identification number. This data set is `test.csv` and this is used to make predictions. Those predictions should be similar with the provided `gendermodel.csv`.

We also have to take a look of the data description provided on __[contest dedicated page](https://www.kaggle.com/c/titanic/data)__:

```
VARIABLE DESCRIPTIONS:
survival Survival
(0 = No; 1 = Yes)
pclass Passenger Class
(1 = 1st; 2 = 2nd; 3 = 3rd)
name Name
sex Sex
age Age
sibsp Number of Siblings/Spouses Aboard
parch Number of Parents/Children Aboard
ticket Ticket Number
fare Passenger Fare
cabin Cabin
embarked Port of Embarkation
(C = Cherbourg; Q = Queenstown; S = Southampton)

SPECIAL NOTES:
Pclass is a proxy for socio-economic status (SES)
1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower

Age is in Years; Fractional if Age less than One (1)
If the Age is Estimated, it is in the form xx.5

With respect to the family relation variables (i.e. sibsp and parch)
some relations were ignored. The following are the definitions used
for sibsp and parch.

Sibling: Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
Spouse: Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances Ignored)
Parent: Mother or Father of Passenger Aboard Titanic
Child: Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic

Other family relatives excluded from this study include cousins,
nephews/nieces, aunts/uncles, and in-laws. Some children travelled
only with a nanny, therefore parch=0 for them. As well, some
travelled with very close friends or neighbors in a village, however,
the definitions do not support such relations.
```

The first step in our adventure is to download those 3 data file in *csv* format. You can do it from __[data section](https://www.kaggle.com/c/titanic/data)__ of the competition. Let's suppose you downloaded somewhere in a local folder. We will name this folder `data` folder, and actually it can have any name you would like.

### 1.2 Read train data from csv file

Because the data is small we can load the whole data in memory with no problems.

Let's see how we can load the data into memory. In rapaio the sets of data are loaded into the form of *frames* (`rapaio.data.Frame`). A frame is basically a tabular data, with columns for each variable (feature) and rows for each instance (in our case for each passenger).


A first try of loading the train data set and see what has happened is the following:


In [2]:
String urlTrain = "https://raw.githubusercontent.com/padreati/rapaio-notebooks/master/data/titanic/train.csv";
String urlTest = "https://raw.githubusercontent.com/padreati/rapaio-notebooks/master/data/titanic/test.csv";
Csv.instance().readUrl(urlTrain).printSummary();

Frame Summary
* rowCount: 891
* complete: 183/891
* varCount: 12
* varNames: 

 0. PassengerId : dbl |  4.         Sex : nom |  8.      Ticket : nom | 
 1.    Survived : bin |  5.         Age : dbl |  9.        Fare : dbl | 
 2.      Pclass : dbl |  6.       SibSp : dbl | 10.       Cabin : nom | 
 3.        Name : nom |  7.       Parch : dbl | 11.    Embarked : nom | 

* summary: 
 PassengerId [dbl]        Survived [bin]    Pclass [dbl]     
      Min. :   1.0000000 0 :         549    Min. : 1.0000000 
   1st Qu. : 223.5000000 1 :         342 1st Qu. : 2.0000000 
    Median : 446.0000000 NAs :         0  Median : 3.0000000 
      Mean : 446.0000000                    Mean : 2.3086420 
   2nd Qu. : 668.5000000                 2nd Qu. : 3.0000000 
      Max. : 891.0000000                    Max. : 3.0000000 
                                                             

                                                   Name [nom]      Sex [nom]       Age [dbl]       
"Jacobsohn, Mrs. Si

How can we interpret the output of the frame's summary?

* We loaded a frame which has $891$ rows and $12$ columns (variables)
* From all the rows, $183$ are complete (non missing data)
* The name of the variables are listed, together with their types
* It follows a data summary for the frame: 6 number summary for numeric variables, most frequent levels for nominal variables

Let's inspect each variable and see how it fits our needs.

**PassengedId**

The type for this variable is index (integer values). This field looks like an identifier for the passenger, so from our point of view the sorting is not required. What we can do, but is not required, is to change the field type to nominal. Anyway, we do not need this field for learning since it should be unique for each instance, thus the predictive power is null. We will ignore it for now since we will not consider it for learning

**Survived**

This is our target variable. It is parsed as binary, but since we do classification, we will change it's type to nominal. We do that directly from the csv parsing, by indicating that we want Survived parsed as nominal variable:


In [3]:
Csv.instance()
.withTypes(VType.NOMINAL, "Survived")
.readUrl(urlTrain)
.printSummary();

Frame Summary
* rowCount: 891
* complete: 183/891
* varCount: 12
* varNames: 

 0. PassengerId : dbl |  4.         Sex : nom |  8.      Ticket : nom | 
 1.    Survived : nom |  5.         Age : dbl |  9.        Fare : dbl | 
 2.      Pclass : dbl |  6.       SibSp : dbl | 10.       Cabin : nom | 
 3.        Name : nom |  7.       Parch : dbl | 11.    Embarked : nom | 

* summary: 
 PassengerId [dbl]        Survived [nom]    Pclass [dbl]     
      Min. :   1.0000000       0 :   549    Min. : 1.0000000 
   1st Qu. : 223.5000000       1 :   342 1st Qu. : 2.0000000 
    Median : 446.0000000                  Median : 3.0000000 
      Mean : 446.0000000                    Mean : 2.3086420 
   2nd Qu. : 668.5000000                 2nd Qu. : 3.0000000 
      Max. : 891.0000000                    Max. : 3.0000000 
                                                             

                                                   Name [nom]      Sex [nom]       Age [dbl]       
"Jacobsohn, Mrs. Si

And notice how type of the `Survived` variable changed to nominal.


**Pclass**

This variable has index type. We can keep it like it is or we can change it to nominal. Both ways can be useful. For example parsed as index could give an interpretation to the order. We can say that somehow, because of ordering class 1 is lower than class 2, and class 2 is between classes 1 and 3. At the same time we can keep it as nominal if we do not want to use the ordering. Let's choose nominal for now, considering that 1,2 and 3 are just labels for type of tickets, with no other meaning attached. We proceed in the same way:

In [4]:
Csv.instance()
.withTypes(VType.NOMINAL, "Survived", "Pclass")
.readUrl(urlTrain)
.printSummary();

Frame Summary
* rowCount: 891
* complete: 183/891
* varCount: 12
* varNames: 

 0. PassengerId : dbl |  4.         Sex : nom |  8.      Ticket : nom | 
 1.    Survived : nom |  5.         Age : dbl |  9.        Fare : dbl | 
 2.      Pclass : nom |  6.       SibSp : dbl | 10.       Cabin : nom | 
 3.        Name : nom |  7.       Parch : dbl | 11.    Embarked : nom | 

* summary: 
 PassengerId [dbl]        Survived [nom]  Pclass [nom] 
      Min. :   1.0000000       0 :   549     3 :   491 
   1st Qu. : 223.5000000       1 :   342     1 :   216 
    Median : 446.0000000                     2 :   184 
      Mean : 446.0000000                               
   2nd Qu. : 668.5000000                               
      Max. : 891.0000000                               
                                                       

                                                   Name [nom]      Sex [nom]       Age [dbl]       
"Jacobsohn, Mrs. Sidney Samuel (Amy Frances Christy)" :     1   mal

Notice that we append the variable name after `Survived`. This is possible since the `withTypes` method specify a type, and follows a dynamic array of strings, for the names of variables.

**Name**

This is the passenger names and the values are unique. As it is, the predictive power of this field is null. We keep it as it is. Note that it contains valuable information, but not in this direct form.

**Sex**

This field specifies the gender of the passenger. We have $577$ males and $314$ females.

**Age**

This field specifies the age of an passenger. We would expect that to parse this variable as numeric or at leas index, but is nominal. Why that happened? Notice that the values looks like numbers. But the first value (the most frequent one, $117$ instances) has nothing specified. Well, the variable is nominal has to do with how *Csv* parsing handles missing values. By default, the *csv* parsing considers as missing values only the string "?". But the most frequent value in this field is empty string "". This means that empty string is not considered a missing value. Because empty string can't produce numbers from parsing, the variable is *promoted* to nominal.

We can customize the missing value handling by specifying the valid strings for that purpose. We use `.useNAValues(String...naValues)` to tell the parser all the valid strings which are missing values. In our case we want just the empty string to be a missing value. When the parser will found an empty string it will set the variable value as missing value. It will *not promote* variable to nominal, since a missing value is a legal value.

In [5]:
Csv.instance()
.withNAValues("")
.withTypes(VType.NOMINAL, "Survived", "Pclass")
.readUrl(urlTrain)
.printSummary();

Frame Summary
* rowCount: 891
* complete: 183/891
* varCount: 12
* varNames: 

 0. PassengerId : dbl |  4.         Sex : nom |  8.      Ticket : nom | 
 1.    Survived : nom |  5.         Age : dbl |  9.        Fare : dbl | 
 2.      Pclass : nom |  6.       SibSp : dbl | 10.       Cabin : nom | 
 3.        Name : nom |  7.       Parch : dbl | 11.    Embarked : nom | 

* summary: 
 PassengerId [dbl]        Survived [nom]  Pclass [nom] 
      Min. :   1.0000000       0 :   549     3 :   491 
   1st Qu. : 223.5000000       1 :   342     1 :   216 
    Median : 446.0000000                     2 :   184 
      Mean : 446.0000000                               
   2nd Qu. : 668.5000000                               
      Max. : 891.0000000                               
                                                       

                                                   Name [nom]      Sex [nom]       Age [dbl]       
"Jacobsohn, Mrs. Sidney Samuel (Amy Frances Christy)" :     1   mal

Notice what happened: *Age* field is now numeric and it contains $177$ missing values.

**SibSp**

It's meaning is "siblings/spouses". It's parsed as index, which is natural. In pathological cases with sick imagination we can consider a "quarter of a wife" for example.

**Parch**

It's meaning is "parents/children". It is naturally parsed as index.

**Ticket**

This is the code of the ticket. Probably a family can have the same ticket, thus must be the reason why the frequencies have values up to $$7$$. This field is nominal. It has low predictive power used directly. Perhaps contains valuable information, but used directly in row format would not help much.

**Fare**

This is the price for passenger fare and should be numeric, like it is.

**Cabin**

Code of the passenger's cabin, parsed as nominal. Same notes as for `Ticket` variable.

**Embarked**

Code for the embarking city, which could be: C = Cherbourg, Q = Queenstown, S = Southampton. It's parsed as nominal and has $2$ missing values.

If we are content with our parsing, we load data into a data frame for later use:

In [6]:
Frame train = Csv.instance().withNAValues("").withTypes(VType.NOMINAL, "Survived", "Pclass").readUrl(urlTrain);
train.printSummary();

Frame Summary
* rowCount: 891
* complete: 183/891
* varCount: 12
* varNames: 

 0. PassengerId : dbl |  4.         Sex : nom |  8.      Ticket : nom | 
 1.    Survived : nom |  5.         Age : dbl |  9.        Fare : dbl | 
 2.      Pclass : nom |  6.       SibSp : dbl | 10.       Cabin : nom | 
 3.        Name : nom |  7.       Parch : dbl | 11.    Embarked : nom | 

* summary: 
 PassengerId [dbl]        Survived [nom]  Pclass [nom] 
      Min. :   1.0000000       0 :   549     3 :   491 
   1st Qu. : 223.5000000       1 :   342     1 :   216 
    Median : 446.0000000                     2 :   184 
      Mean : 446.0000000                               
   2nd Qu. : 668.5000000                               
      Max. : 891.0000000                               
                                                       

                                                   Name [nom]      Sex [nom]       Age [dbl]       
"Jacobsohn, Mrs. Sidney Samuel (Amy Frances Christy)" :     1   mal

### 1.3 Read test data from *csv* file

Once we have a training frame we can load also the test data. We do that to take a look at the frame and because data is small and there is no memory or time problem cost associated with it. To avoid adding again the *csv* options and to get identical levels nominal variables, we use a different way to parse the data set. We specify variable types by frame templates:

In [7]:
Frame test = Csv.instance().withNAValues("").withTemplate(train).readUrl(urlTest);

Instead to specify again the preferred types for variables, we use train frame as a template for variable types. This has also the side effect that the encoding of categorical variables is identical.

In [8]:
test.printSummary();

Frame Summary
* rowCount: 418
* complete: 87/418
* varCount: 11
* varNames: 

 0. PassengerId : dbl |  4.         Age : dbl |  8.        Fare : dbl | 
 1.      Pclass : nom |  5.       SibSp : dbl |  9.       Cabin : nom | 
 2.        Name : nom |  6.       Parch : dbl | 10.    Embarked : nom | 
 3.         Sex : nom |  7.      Ticket : nom | 

* summary: 
 PassengerId [dbl]          Pclass [nom]                                   Name [nom]      Sex [nom] 
      Min. :   892.0000000     3 :   218                "Birnbaum, Mr. Jakob" :     1   male :   266 
   1st Qu. :   996.2500000     1 :   107 "Willer, Mr. Aaron (Abi Weller"")""" :     1 female :   152 
    Median : 1,100.5000000     2 :    93          "Olsson, Mr. Oscar Wilhelm" :     1                
      Mean : 1,100.5000000                               "Barry, Miss. Julia" :     1                
   2nd Qu. : 1,204.7500000                                 "Thomas, Mr. John" :     1                
      Max. : 1,309.0000000   

We can note that we don't have *Survived* variable anymore. This is correct since this is what we have to predict. Note also that the types for the remaining variables are the same with training data set.

## 2. Build simple models

### 2.1 Build a majority model

To make a first submission we will build a very simple model, which classifies with a single value all instances. This value is the majority label.

Let's inspect at how target variable look like.

In [9]:
DensityVector.fromLevelCounts(false, train.rvar("Survived")).printSummary();

  0   1 
  -   - 
549 342 



As we already new from the summary, the number of passengers who didn't survived is lower than those who did. Let's see percentages:

In [10]:
DensityVector.fromLevelCounts(false, train.rvar("Survived")).normalize().printSummary();

        0         1 
        -         - 
0.6161616 0.3838384 



A similar result can be obtined using group functionality.

In [11]:
Group.from(train, "Survived").aggregate(Group.count(1, "Survived")).printContent();

group by: Survived
group count: 2
group by functions: GroupByFunction{name=count,varNames=[Survived]}

    Survived Survived_count_N1  
[0]        0 0.6161616161616161 
[1]        1 0.3838383838383838 




We note that there are about $61\%$ of passengers who did not survived. We will create a submit data set, which we will save for later submission. How we can do that?

In [12]:
VarNominal prediction = VarNominal.from(test.rowCount(), row -> "0").withName("Survived");
Frame submit = SolidFrame.byVars(test.rvar("PassengerId"), prediction);

Csv.instance().withQuotes(false).write(submit, "majority_submit.csv");

In the first line we created a new nominal variable. The size of the new variable is the number of rows from the test frame. For each row we produce the same label `"0"`. We name this variable `Survived`.

In the second line we created a new frame taking the variable named `PassengerId` from the test data set and the new prediction variable.

In the last line we wrote a new csv file with the csv parsing utility, taking care to not write quotes. We can submit this file and see which are the results.

![Submission result with majority classifierModel](images/titanic-majority-submit.png)

### 2.2 Build a simple gender model

It has been said that "women and children first" really happened during Titanic tragedy. If this was true or not, we do not know. But we can use data to see if we are hearing the same story. For now we will take the gender and see if it had an influence. We will build a contingency table for variables `Sex` and `Survived`.

In [13]:
DensityTable.fromLevelCounts(false, train.rvar("Sex"), train.rvar("Survived")).printSummary();

         0   1 total 
  male 468 109  577  
female  81 233  314  
 total 549 342  891  



On rows we have levels of `Sex` variable. On columns we have levels of `Sex` variable. Cells are computed as counts. What we see is that there are a lot of men who did not survived and a lot of women who does. We will normalize on rows to take a closer look.

In [14]:
DensityTable.fromLevelCounts(false, train.rvar("Sex"), train.rvar("Survived")).normalizeOnRows().printSummary();

               0         1 total 
  male 0.8110919 0.1889081   1   
female 0.2579618 0.7420382   1   
 total 1.0690536 0.9309464   2   



It seems that men survived with a rate of $0.19$ and women with $0.74$. The values are so obvious, we need no hypothesis testing to check that this variable is significant for classification. We will build a simple model where we predict as survived all the women and not survived all the men.

In [15]:
Var prediction = VarNominal.from(test.rowCount(), row -> test.getLabel(row, "Sex").equals("male") ? "0" : "1").withName("Survived");
Frame submit = SolidFrame.byVars(test.rvar("PassengerId"), prediction);
Csv.instance().withQuotes(false).write(submit, "gender_submit.csv");

__![Submission result with gender classifierModel](images/titanic-gender-submit.png)__

## 3. Tree model

Building models in the manual way is often not the way to go. This process is tedious and time consuming. There are already built automated procedures, which incorporate miscellaneous approaches to learn a classifierModel. One of the often used models is the decision tree. Decision trees are greedy local approximations build in a recursive greedy fashion. Often the split decision at node level uses a single feature. At leave nodes a simple majority classifierModel creates the classification rule.

### 3.1 Gender model with decision tree
Initially we will build a CART decision tree using as input feature the *Sex* variable. We do this to exemplify how a manual rule can be created in an automated fashion.

In [16]:
Frame tr = train.mapVars("Survived,Sex");
CTree tree = CTree.newCART();
tree.fit(tr, "Survived");
tree.printSummary();

CTree model

Description:
CTree{purity=GiniGain,splitter=Weighted,varSelector=VarSelector[ALL]}

Capabilities:
types inputs/targets: NOMINAL,INT,DOUBLE,BINARY/NOMINAL
counts inputs/targets: [1,1000000] / [1,1]
missing inputs/targets: true/false

Learned model:

total number of nodes: 3
total number of leaves: 2
description:
split, n/err, classes (densities) [* if is leaf / purity if not]

|- 0. root    891/342 0 (0.616 0.384 ) [0.139648]
|   |- 1. Sex = 'female'    314/81 1 (0.258 0.742 ) *
|   |- 2. Sex != 'female'    577/109 0 (0.811 0.189 ) *



Taking a closer look at the last three rows from the output, one can identify our manual rule. Basically the interpretation is: *"all the females survived, all the males did not"*. For exemplification purposes we build also the submit file.

In [17]:
// do a cross validation
ClassifierEvaluation.cv(tr, "Survived", tree, 10, Accuracy.newMetric(true)).run().printContent();

Model:
CTree{purity=GiniGain,splitter=Weighted,varSelector=VarSelector[ALL]}
CV score in training data
    dataset  metric    mean       std    
[0]    test Accuracy 0.7867915 0.0360383 
[1]   train Accuracy 0.7867569 0.0040034 




In [18]:
// fit the tree to test data frame
ClassifierResult pred = tree.predict(test);
// build teh submission
Frame submit = SolidFrame.byVars(test.rvar("PassengerId"),pred.firstClasses().withName("Survived"));
// write to a submit file
Csv.instance().withQuotes(false).write(submit, "tree1-model.csv");

### 3.2 Enrich tree by using other features

Our training data set has more than a single input feature. Thus We can state we didn't use all the information available. We will add now the class and embarking port and see how it behaves.

In [19]:
Frame tr = train.mapVars("Survived,Sex,Pclass,Embarked");

CTree tree = CTree.newCART();
tree.fit(tr, "Survived");
tree.printSummary();

ClassifierEvaluation.cv(tr, "Survived", tree, 10, Accuracy.newMetric(true)).run().printContent();

ClassifierResult pred = tree.predict(test);
Frame submit = SolidFrame.byVars(test.rvar("PassengerId"),pred.firstClasses().withName("Survived"));
Csv.instance().withQuotes(false).write(submit, "tree2-model.csv");

CTree model

Description:
CTree{purity=GiniGain,splitter=Weighted,varSelector=VarSelector[ALL]}

Capabilities:
types inputs/targets: NOMINAL,INT,DOUBLE,BINARY/NOMINAL
counts inputs/targets: [1,1000000] / [1,1]
missing inputs/targets: true/false

Learned model:

total number of nodes: 17
total number of leaves: 9
description:
split, n/err, classes (densities) [* if is leaf / purity if not]

|- 0. root    891/342 0 (0.616 0.384 ) [0.139648]
|   |- 1. Sex = 'female'    314/81 1 (0.258 0.742 ) [0.0204665]
|   |   |- 3. Pclass = '2'    76/6 1 (0.079 0.921 ) [0.0003369]
|   |   |   |- 7. Embarked = 'Q'    2/0 1 (0 1 ) *
|   |   |   |- 8. Embarked != 'Q'    74/6 1 (0.081 0.919 ) [0.0013737]
|   |   |   |   |- 13. Embarked = 'C'    7/0 1 (0 1 ) *
|   |   |   |   |- 14. Embarked != 'C'    67/6 1 (0.09 0.91 ) *
|   |   |- 4. Pclass != '2'    238/75 1 (0.315 0.685 ) [0.0009488]
|   |   |   |- 9. Embarked = 'Q'    36/9 1 (0.262 0.738 ) *
|   |   |   |- 10. Embarked != 'Q'    204/66 1 (0.324 0.676 

The tree is much richer and there are more chances to be better. This is what happened after submission.

__![Results after submission of enriched tree](images/titanic-tree2-submit.png)__

**Nice!**. We advanced $704$ positions and improved our score with $0.01435$. On public leader board we have a nice $0.77990$ accuracy score.

### 3.3 Overfitting with trees

What about using other input features to improve our prediction accuracy? There are some of them which we can include directly, with no changes: *Age*,*Fare*,*SibSp* and *Parch*.

We can change the script slightly, to include those new input features. But we can do better, we can use cross-validation to estimate what will happen.

In [20]:
CTree tree = CTree.newCART();
ClassifierEvaluation.cv(train.mapVars("Survived,Sex,Pclass,Embarked"),"Survived", tree, 10, Accuracy.newMetric(true)).run().printContent();
ClassifierEvaluation.cv(train.mapVars("Survived,Sex,Pclass,Embarked,Age,Fare,SibSp,Parch"),"Survived", tree, 10, Accuracy.newMetric(true)).run().printContent();;

Model:
CTree{purity=GiniGain,splitter=Weighted,varSelector=VarSelector[ALL]}
CV score in training data
    dataset  metric    mean       std    
[0]    test Accuracy 0.7867291 0.0368467 
[1]   train Accuracy 0.7867561 0.0040915 


Model:
CTree{purity=GiniGain,splitter=Weighted,varSelector=VarSelector[ALL]}
CV score in training data
    dataset  metric    mean       std    
[0]    test Accuracy 0.8192385 0.0385895 
[1]   train Accuracy 0.9196911 0.0068803 




Notice that the 10-crossfold estimator of the accuracy has dropped with a large quantity. What happens? We can have an idea if we take a look at the learned tree:

In [21]:
tree.fit(train.mapVars("Survived,Sex,Pclass,Embarked,Age,Fare,SibSp,Parch"), "Survived");
tree.printSummary();

CTree model

Description:
CTree{purity=GiniGain,splitter=Weighted,varSelector=VarSelector[ALL]}

Capabilities:
types inputs/targets: NOMINAL,INT,DOUBLE,BINARY/NOMINAL
counts inputs/targets: [1,1000000] / [1,1]
missing inputs/targets: true/false

Learned model:

total number of nodes: 391
total number of leaves: 196
description:
split, n/err, classes (densities) [* if is leaf / purity if not]

|- 0. root    891/342 0 (0.616 0.384 ) [0.139648]
|   |- 1. Sex = 'female'    314/81 1 (0.258 0.742 ) [0.0282131]
|   |   |- 3. Fare <= 48.2    225/76 1 (0.338 0.662 ) [0.0645841]
|   |   |   |- 7. Pclass = '2'    74/6 1 (0.081 0.919 ) [0.0099206]
|   |   |   |   |- 15. Age <= 56    73/5 1 (0.069 0.931 ) [0.0036241]
|   |   |   |   |   |- 29. Age <= 23.5    22/0 1 (0 1 ) *
|   |   |   |   |   |- 30. Age > 23.5    53/5 1 (0.095 0.905 ) [0.0031376]
|   |   |   |   |   |   |- 51. Fare <= 26.125    45/5 1 (0.112 0.888 ) [0.0044159]
|   |   |   |   |   |   |   |- 73. Fare <= 12.825    8/0 1 (0 1 ) *
| 


Notice how large is the tree. Basically the tree was full grown and overfit the training data set too much. We can ask ourselves why that happens? Why it happens now, and did not happened when we had fewer inputs? The answer is that it happened also before. But it's consequences were not so drastic.

The first tree used for training just $3$ input nominal features. Notice that all three features are nominal. The maximum number of groups which one can form is given by the product of the number of levels for each feature. This total maximal number is $2*3*3=18$. It practically exhausted the discrimination potential of those features. It did overfit in that reduced space of features. When we apply the model to the whole data set, the effect of exhaustion is not seen anymore.

The second tree does the same thing, but this time in a richer space, with added input dimensions. Compared with the full feature space, we see the effect.

There are two approaches to avoid overfit for a decision tree. The first approach is to stop learning up to the moment when we exhaust the data. The name for this approach is *early stop*. We can do that by specifying some parameters of the tree model:

* Set a minimum number of instances for leaf node
* Set a maximal depth for the tree
* Not implemented yet, but easy to do: complexity threshold, maximal number of nodes in a tree

The second approach is to prune the tree. Pruning procedure consists of growing the full tree and later on removing some nodes if they do not provide some type of gain. Currently we implemented only *reduced error pruning strategy*.

We will test with 10-fold cross validation an early-stopping strategy to see how it works.

In [22]:
ClassifierEvaluation.cv(train.mapVars("Survived,Sex,Pclass,Embarked,Age,Fare,SibSp,Parch"),"Survived",
                        tree.maxDepth.set(8).minCount.set(4), 10, Accuracy.newMetric(true)).run().printContent();

Model:
CTree{maxDepth=8,minCount=4,purity=GiniGain,splitter=Weighted,varSelector=VarSelector[ALL]}
CV score in training data
    dataset  metric    mean       std    
[0]    test Accuracy 0.8036205 0.0384572 
[1]   train Accuracy 0.8563412 0.0037073 





I tried some values, just to show that we can do something about it, but the progress did not appear. We should try a different approach, and that is an ensemble. Next session contains directions on how to build such an ensemble.

## 4. CForest model

Random forests are well-known to work well when the irreducible error from the training data is high. This is probably the case of this Titanic data set. We have reasons to believe that this is the situation since it was a tragedy. A lot of random or not-so-expected things happened. That happened despite of the bravery and the sacrifice of the crew and others.

Random forests are the invention of [Leo Breiman](https://en.wikipedia.org/wiki/Leo_Breiman). The first design was a joint effort together with [Adele Cutler](http://www.math.usu.edu/adele/). The base of random forests is bagging (or **b**ootstrapp **ag**gregation). On top of that, selecting just a random limited number of variables at each node is the core of the algorithm.

We will work with random forests for now. This ensemble is mode robust and is capable of obtaining much better results than a single tree. At the same time we will introduce 10-fold cross validation to check our progress and estimate the error produced.

In the beginning we will use 10-fold cross validation for estimating the accuracy on public leader board.

We will test first with 10 fold cv the tree model.

In [23]:
ClassifierEvaluation.cv(train.mapVars("Survived,Sex,Pclass,Embarked"), "Survived", CTree.newCART(), 10, Accuracy.newMetric(true)).run().printContent();

Model:
CTree{purity=GiniGain,splitter=Weighted,varSelector=VarSelector[ALL]}
CV score in training data
    dataset  metric    mean       std    
[0]    test Accuracy 0.786779  0.0345536 
[1]   train Accuracy 0.7867567 0.0038363 




### 4.1 Our first random forest

The name of the random forest implementation is `CForest`. To build a new ensemble of trees, one have to instantiate it in the following way:

In [24]:
CForest rf = CForest.newModel();

There are a lot of things which can be customized for a random forest. Among them one can change:

* Number of trees for classification
* Which kind of weak classifierModel to use (you can customize this customized accordingly, like any other classifierModel)
* Number of threads in pool (if you want to use parallelism)
* What to do after each running step

Let's build one and use ore new cross validation procedure to estimate it's error.

In [25]:
RandomSource.setSeed(123);
Frame tr = train.mapVars("Survived,Sex,Pclass,Embarked");
CForest rf = CForest.newModel().runs.set(100);
ClassifierEvaluation.cv(tr, "Survived", rf, 10, Accuracy.newMetric(true)).run().printContent();

Model:
CForest{rowSampler=Bootstrap(p=1),runs=100}
CV score in training data
    dataset  metric    mean       std    
[0]    test Accuracy 0.7856679 0.0605696 
[1]   train Accuracy 0.7923679 0.0111615 




Well, an identical output. This is due to the fact that our variables are already exhausted by the tree. It looks like an underfit. If one consider bias variance trade off, one can see this as high bias. We need to enrich our feature space to improve our performance.

Let's be direct and test what would happen if we would use all our directly usable features? This time we will fit also the training data set, to see the distribution of the training error.

In [26]:
RandomSource.setSeed(123);
Frame tr = train.mapVars("Survived,Sex,Pclass,Embarked,Age,Fare,SibSp,Parch");
CForest rf = CForest.newModel().runs.set(100);
ClassifierEvaluation.cv(tr, "Survived", rf, 10, Accuracy.newMetric(true)).run().printContent();

rf.fit(tr, "Survived");
ClassifierResult fit = rf.predict(test);
Confusion.from(tr.rvar("Survived"), rf.predict(tr).firstClasses()).printSummary();

Model:
CForest{rowSampler=Bootstrap(p=1),runs=100}
CV score in training data
    dataset  metric    mean       std    
[0]    test Accuracy 0.8237578 0.0600302 
[1]   train Accuracy 0.9315376 0.0046453 


> Confusion matrix
 - Frequency table
Ac\Pr |    0    1 | total 
----- |    -    - | ----- 
    0 | >530   19 |   549 
    1 |   40 >302 |   342 
----- |    -    - | ----- 
total |  570  321 |   891 
 - Probability table
Ac\Pr |      0      1 | total 
----- |      -      - | ----- 
    0 | >0.595  0.021 | 0.616 
    1 |  0.045 >0.339 | 0.384 
----- |      -      - | ----- 
total |  0.640  0.360 | 1.000 


Complete cases 891 from 891
Acc: 0.9337823         (Accuracy )
F1:  0.9472744         (F1 score / F-measure)
MCC: 0.8594619         (Matthew correlation coefficient)
Pre: 0.9298246         (Precision)
Rec: 0.9653916         (Recall)
G:   0.9474412         (G-measure)



This time we have a good example of overfit. Why is that? Look at the confusion matrix on the training set. We fit too well the training data. This data set is well known for its high irreducible error. And there is an explanation for that. During the tragic event a lot of exceptional things happened. For example I read somewhere that an old lady which had a dog was not allowed to embark with her pet due to regulations. As a consequence she decided to not leave it and she chose to die with him. It's close to impossible to learn those kind of things, even if the information would be available. 

We should reduce the error somehow. We can try to decrease the overfit by adding more learners. Let's see if that would be enough for our purpose.

In [27]:
RandomSource.setSeed(123);
Frame tr = train.mapVars("Survived,Sex,Pclass,Embarked,Age,Fare,SibSp,Parch");
CForest rf = CForest.newModel().runs.set(500);
ClassifierEvaluation.cv(tr, "Survived", rf, 10, Accuracy.newMetric(true)).run().printContent();

rf.fit(tr, "Survived");
ClassifierResult fit = rf.predict(test);
Confusion.from(tr.rvar("Survived"), rf.predict(tr).firstClasses()).printSummary();

Model:
CForest{rowSampler=Bootstrap(p=1),runs=500}
CV score in training data
    dataset  metric    mean       std    
[0]    test Accuracy 0.8248689 0.0603189 
[1]   train Accuracy 0.931288  0.0027264 


> Confusion matrix
 - Frequency table
Ac\Pr |    0    1 | total 
----- |    -    - | ----- 
    0 | >528   21 |   549 
    1 |   41 >301 |   342 
----- |    -    - | ----- 
total |  569  322 |   891 
 - Probability table
Ac\Pr |      0      1 | total 
----- |      -      - | ----- 
    0 | >0.593  0.024 | 0.616 
    1 |  0.046 >0.338 | 0.384 
----- |      -      - | ----- 
total |  0.639  0.361 | 1.000 


Complete cases 891 from 891
Acc: 0.9304153         (Accuracy )
F1:  0.9445438         (F1 score / F-measure)
MCC: 0.8522322         (Matthew correlation coefficient)
Pre: 0.9279438         (Precision)
Rec: 0.9617486         (Recall)
G:   0.944695         (G-measure)



This is slightly better than before. But the difference does not look significantly better than previous. We will use a simple pre-pruning strategy is to limit the number instances in leaf nodes. We set the minimum count to $3$.

In [28]:
RandomSource.setSeed(123);
Frame tr = train.mapVars("Survived,Sex,Pclass,Embarked,Age,Fare,SibSp,Parch");
CForest rf = CForest.newModel().model.set(CTree.newCART().minCount.set(3)).runs.set(100);
ClassifierEvaluation.cv(tr, "Survived", rf, 10, Accuracy.newMetric(true)).run().printFullContent();

rf.fit(tr, "Survived");
ClassifierResult fit = rf.predict(test);
Confusion.from(tr.rvar("Survived"), rf.predict(tr).firstClasses()).printSummary();

Model:
CForest{model=CTree,rowSampler=Bootstrap(p=1),runs=100}
Raw scores:
     dataset round fold Accuracy       dataset round fold Accuracy       dataset round fold Accuracy  
 [0]    test     0    0 0.8555556  [7]    test     0    7 0.741573  [14]   train     0    4 0.8952618 
 [1]    test     0    1 0.7865169  [8]    test     0    8 0.7865169 [15]   train     0    5 0.8965087 
 [2]    test     0    2 0.8988764  [9]    test     0    9 0.7640449 [16]   train     0    6 0.90399   
 [3]    test     0    3 0.8426966 [10]   train     0    0 0.8963795 [17]   train     0    7 0.9077307 
 [4]    test     0    4 0.8539326 [11]   train     0    1 0.8990025 [18]   train     0    8 0.9014963 
 [5]    test     0    5 0.8988764 [12]   train     0    2 0.8952618 [19]   train     0    9 0.9002494 
 [6]    test     0    6 0.752809  [13]   train     0    3 0.8902743 

Round scores:
    dataset round Accuracy_mean Accuracy_std 
[0]    test     0   0.8181398    0.0560159   
[1]   train     0   0.898615

Notice that we changed the classifierModel used by `CForest`. This is the same classifierModel used by default by random forest. We do this because we customized the classifierModel by changing the min count parameter.

That had indeed some effect. However after submitting to competition we did not saw any improvement. We should look forward to engineer a little bit our features for further improvements.

## 5. Feature engineering

### 5.1 Title feature

It is clear that we can't use directly the `"Name"` variable. This is due to the fact that names are almost unique, and that leads to a tiny generalization power. To understand that we should see that even if we learned that a passenger with a given name survived or not. We can't decide if another passenger survived, using only the name of the new passenger.

Lets inspect some of the values from `"Name"` variable.

In [29]:
train.rvar("Name").printContent();

VarNominal [name:"Name", rowCount:891]
 row                             value                            
  [0]                  "Braund, Mr. Owen Harris"                  
  [1]    "Cumings, Mrs. John Bradley (Florence Briggs Thayer)"    
  [2]                  "Heikkinen, Miss. Laina"                   
  [3]       "Futrelle, Mrs. Jacques Heath (Lily May Peel)"        
  [4]                 "Allen, Mr. William Henry"                  
  [5]                     "Moran, Mr. James"                      
  [6]                  "McCarthy, Mr. Timothy J"                  
  [7]              "Palsson, Master. Gosta Leonard"               
  [8]     "Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)"     
  [9]            "Nasser, Mrs. Nicholas (Adele Achem)"            
 [10]              "Sandstrom, Miss. Marguerite Rut"              
 [11]                 "Bonnell, Miss. Elizabeth"                  
 [12]              "Saundercock, Mr. William Henry"               
 [13]                "A

We notice that the names contains title information of the individual. This is valuable, but how can we benefit from that? First of all see that the format of that string is clear: space + title + dot + space. We can try to model a regular expression or we can take a simpler, but manual path. Intuition tells us that there should not be too many keys.

We build a set with known keys. After that we filter out names with known titles, and print first twenty of them. We see that we have already `"Mrs"` and `"Mr"`. Let's find others.

In [30]:
// build incrementally a set with known keys
HashSet<String> keys = new HashSet<>();
keys.add("Mrs");
keys.add("Mr");

// filter out names with known keys
// print first twenty to inspect and see other keys
train.rvar("Name").stream().mapToString().filter(txt -> {
    for(String key : keys)
        if(txt.contains(" " + key + ". "))
            return false;
    return true;
}).limit(20).forEach(WS::println);

"Heikkinen, Miss. Laina"
"Palsson, Master. Gosta Leonard"
"Sandstrom, Miss. Marguerite Rut"
"Bonnell, Miss. Elizabeth"
"Vestrom, Miss. Hulda Amanda Adolfina"
"Rice, Master. Eugene"
"McGowan, Miss. Anna ""Annie"""
"Palsson, Miss. Torborg Danira"
"O'Dwyer, Miss. Ellen ""Nellie"""
"Uruchurtu, Don. Manuel E"
"Glynn, Miss. Mary Agatha"
"Vander Planke, Miss. Augusta Maria"
"Nicola-Yarred, Miss. Jamila"
"Laroche, Miss. Simonne Marie Anne Andree"
"Devaney, Miss. Margaret Delia"
"O'Driscoll, Miss. Bridget"
"Panula, Master. Juha Niilo"
"Rugg, Miss. Emily"
"West, Miss. Constance Mirium"
"Goodwin, Master. William Frederick"


We reduced our search and found other titles like `"Miss"`, `"Master"`. We arrive at the following set of keys:

In [31]:
HashSet<String> keys = new HashSet<>();
keys.addAll(Arrays.asList("Mrs", "Mme", "Lady", "Countess", "Mr", "Sir","Don", "Ms", "Miss", 
"Mlle", "Master", "Dr","Col", "Major", "Jonkheer", "Capt", "Rev"));

VarNominal title = train.rvar("Name").stream().mapToString().map(txt -> {
    for(String key : keys)
        if(txt.contains(" " + key + ". "))
            return key;
    return "?";
}).collect(VarNominal.collector());
DensityVector.fromLevelCounts(true, title).printSummary();

?  Mr Mrs Miss Master Don Rev Dr Mme Ms Major Lady Sir Mlle Col Capt Countess Jonkheer 
-  -- --- ---- ------ --- --- -- --- -- ----- ---- --- ---- --- ---- -------- -------- 
0 517 125 182    40    1   6  7   1  1    2    1    1   2    2   1      1        1     



We note that we exhausted training data. This is enough. It is possible that in test data to appear new titles. We will consider them missing values. That is why we return `"?"` when no matching is found. Another thing to notice is that some of the labels have few number of appearances. We will merge them in a greater category.

Another useful feature built in `rapaio` is filters. There are two types of filters: variable filters and frame filters. The nice part of frame filters is that learning algorithms are able to use frame filters naturally, in order to make feature transformations on data. This kind of filters are called *input filters* from the learning algorithm perspective. It is important that you know that input filters transforms features before train phase and also on fit phase.

We will build a learning filter to create a new feature.

In [32]:
class TitleFilter implements FFilter {
    private HashMap<String, String[]> replaceMap = new HashMap<>();
    private Function<String, String> titleFun = txt -> {
        for (Map.Entry<String, String[]> e : replaceMap.entrySet()) {
            for (int i = 0; i < e.getValue().length; i++) {
                if (txt.contains(" " + e.getValue()[i] + ". "))
                    return e.getKey();
                }
            }
        return "?";
    };

    @Override
    public void fit(Frame df) {
        replaceMap.put("Mrs", new String[]{"Mrs", "Mme", "Lady", "Countess"});
        replaceMap.put("Mr", new String[]{"Mr", "Sir", "Don", "Ms"});
        replaceMap.put("Miss", new String[]{"Miss", "Mlle"});
        replaceMap.put("Master", new String[]{"Master"});
        replaceMap.put("Dr", new String[]{"Dr"});
        replaceMap.put("Military", new String[]{"Col", "Major", "Jonkheer", "Capt"});
        replaceMap.put("Rev", new String[]{"Rev"});
    }

    @Override
    public Frame apply(Frame df) {
        VarNominal title = VarNominal.empty(0, new ArrayList<>(replaceMap.keySet())).withName("Title");
        df.rvar("Name").stream().mapToString().forEach(name -> title.addLabel(titleFun.apply(name)));
        return df.bindVars(title);
    }
    
    @Override
    public TitleFilter newInstance() {
        return new TitleFilter();
    }
    
    public String[] varNames() {
        return new String[0];
    }
}

Now let's try a new random forest on the reduced data set and also on title.

In [33]:
RandomSource.setSeed(124);

Frame df_train = train.fapply(new TitleFilter());
Frame df_test = test.fapply(new TitleFilter());

CForest rf = CForest.newModel().model.set(CTree.newCART().minCount.set(3)).runs.set(100);

ClassifierEvaluation.cv(df_train.mapVars("Survived,Sex,Pclass,Embarked,Age,Fare,SibSp,Parch"), "Survived", rf, 10, Accuracy.newMetric(true)).run().printContent();

rf.fit(df_train.mapVars("Survived,Sex,Pclass,Embarked,Title"), "Survived");
rf.printSummary();

ClassifierResult fit = rf.predict(df_test);
Confusion.from(train.rvar("Survived"), rf.predict(df_train).firstClasses()).printSummary();

Model:
CForest{model=CTree,rowSampler=Bootstrap(p=1),runs=100}
CV score in training data
    dataset  metric    mean       std    
[0]    test Accuracy 0.8294382 0.0421271 
[1]   train Accuracy 0.8991147 0.0038707 


CForest

Description:
CForest{model=CTree,rowSampler=Bootstrap(p=1),runs=100}

Capabilities:
types inputs/targets: NOMINAL,INT,DOUBLE,BINARY/NOMINAL
counts inputs/targets: [1,1000000] / [1,1]
missing inputs/targets: true/false

Model fitted: true.
Learned model:
input vars: 
0.      Sex : NOMINAL  | 
1.   Pclass : NOMINAL  | 
2. Embarked : NOMINAL  | 
3.    Title : NOMINAL  | 

target vars:
> Survived : NOMINAL [?,0,1]


Fitted trees:100
oob enabled:false

> Confusion matrix
 - Frequency table
Ac\Pr |    0    1 | total 
----- |    -    - | ----- 
    0 | >468   81 |   549 
    1 |  107 >235 |   342 
----- |    -    - | ----- 
total |  575  316 |   891 
 - Probability table
Ac\Pr |      0      1 | total 
----- |      -      - | ----- 
    0 | >0.525  0.091 | 0.616 
    1 | 

Now that looks slightly better than our best classifierModel. We submit that to kaggle to see if there is any improvement.

![Progress after incorporating title name into input features](images/titanic-rf1-model.png)

### 5.2 Other features

There are various authors which published their work on solving this kaggle competition. Most interesting part of their work is the feature engineering section. I developed here some ideas in order to show how one can do this with the library.

#### 5.2.1 Family size

Using directly `"SibSp"` and `"Parch"` fields yields no value for a random forest classifierModel. Studying this two features it looks like those values can be both combined into a single one by summation. This would give us a family size estimator.

In order to have an idea of the performance of this new estimator I used a ChiSquare independence test. The idea is to study if those features taken separately worth less than combined.

In [34]:
// convert sibsp and parch to nominal types to be able to use a chi-square test
Var sibsp = VarNominal.from(train.rowCount(), row -> train.getLabel(row, "SibSp"));
Var parch = VarNominal.from(train.rowCount(), row -> train.getLabel(row, "Parch"));

// test individually each feature
ChiSqIndependence.from(train.rvar("Survived"), sibsp, true).printSummary();
ChiSqIndependence.from(train.rvar("Survived"), parch, true).printSummary();

// build a combined feature by summation, as nominal
VarNominal familySize = VarNominal.from(train.rowCount(),
row -> "" + (1 + train.getInt(row, "SibSp") + train.getInt(row, "Parch")));

// run the chi-square test on sumation
ChiSqIndependence.from(train.rvar("Survived"), familySize, true).printSummary();

> ChiSqIndependence

Pearson's Chi-squared test with Yates' continuity correction

X-squared = 31.9279202, df = 6, p-value = 0.0000168

Observed data:
      1.0 0.0 3.0 4.0 2.0 5.0 8.0 total 
    0  97 398 12  15  15   5   7   549  
    1 112 210  4   3  13   0   0   342  
total 209 608 16  18  28   5   7   891  

Expected data:
              1.0         0.0        3.0        4.0        2.0       5.0       8.0 total 
    0 128.7777778 374.6262626  9.8585859 11.0909091 17.2525253 3.0808081 4.3131313  549  
    1  80.2222222 233.3737374  6.1414141  6.9090909 10.7474747 1.9191919 2.6868687  342  
total 209         608         16         18         28         5         7          891  


> ChiSqIndependence

Pearson's Chi-squared test with Yates' continuity correction

X-squared = 23.3892671, df = 6, p-value = 0.0006761

Observed data:
      0.0 1.0 2.0 5.0 3.0 4.0 6.0 total 
    0 445  53 40   4   2   4   1   549  
    1 233  65 40   1   3   0   0   342  
total 678 118 80   5   5   4   1 

How we can interpret the result? The test says that each feature brings value separately. The *p-value* associated with both test could be considered significant. That means there are string evidence that those features are not independent of target class. As a conclusion, those features are useful. The last test is made for their summation. It looks like he test is more significant than the previous two. As a consequence we can use the summation instead of those two values taken independently.

### 5.3 Cabin and Ticket

It seems that cabin and ticket denominations are not useful as they are. There are some various reasons why not to do so. First of all they have many missing values. But a stringer reason is that both have too many levels to contain solid generalization base for learning.

If we take only the first letter from each of those two fields, more generalization can happen. This is probably due to the fact that perhaps there is some localization information encoded in those. Perhaps information about the deck, the comfort level, auxiliary functions is encoded in there. As a conclusion it worth a try so we proceed with this thing.

To combine all those things we can do it in a single filter or into many filters applied on data. I chose to do a single custom filter to solve all those problems.

In [35]:
class CustomFilter implements FFilter {

    private HashMap<String, String[]> replaceMap = new HashMap<>();
    private Function<String, String> titleFun = txt -> {
        for (Map.Entry<String, String[]> e : replaceMap.entrySet()) {
            for (int i = 0; i < e.getValue().length; i++) {
                if (txt.contains(" " + e.getValue()[i] + ". "))
                    return e.getKey();
            }
        }
        return "?";
    };

    public void fit(Frame df) {
        replaceMap.put("Mrs", new String[]{"Mrs", "Mme", "Lady", "Countess"});
        replaceMap.put("Mr", new String[]{"Mr", "Sir", "Don", "Ms"});
        replaceMap.put("Miss", new String[]{"Miss", "Mlle"});
        replaceMap.put("Master", new String[]{"Master"});
        replaceMap.put("Dr", new String[]{"Dr"});
        replaceMap.put("Military", new String[]{"Col", "Major", "Jonkheer", "Capt"});
        replaceMap.put("Rev", new String[]{"Rev"});
    }

    public Frame apply(Frame df) {
        VarNominal title = VarNominal.empty(0, new ArrayList<>(replaceMap.keySet())).withName("Title");
        df.rvar("Name").stream().mapToString().forEach(name -> title.addLabel(titleFun.apply(name)));

        Var famSize = VarDouble.from(df.rowCount(), row ->
        1.0 + df.getInt(row, "SibSp") + df.getInt(row, "Parch")).withName("FamilySize");

        Var ticket = VarNominal.from(df.rowCount(), row ->
        df.isMissing(row, "Ticket") ? "?" : df.getLabel(row, "Ticket")
        .substring(0, 1).toUpperCase()
        ).withName("Ticket");

        Var cabin = VarNominal.from(df.rowCount(), row ->
        df.isMissing(row, "Cabin") ? "?" : (df.getLabel(row, "Cabin")
        .substring(0, 1).toUpperCase())
        ).withName("Cabin");

        return df.removeVars("Ticket,Cabin").bindVars(famSize, ticket, cabin, title).copy();
    }

    public CustomFilter newInstance() { return new CustomFilter(); }
    
    public String[] varNames() { return new String[0]; }
}

### 5.4 Another try with random forests

So we have some new features and we look to learn from them. We can use a previous classifierModel like random forests to test it before submit.

But we know that we are in danger to overfit is rf. One idea is to transform numeric features into nominal ones by a process named discretization. For this purpose we use a filter from the library called `FQuantileDiscrete`. This filter computes a given number of quantile intervals and put labels according with those intervals on numerical values. Let's see how we proceed and how the data looks like:


In [36]:
FFilter[] inputFilters = new FFilter[]{
new CustomFilter(),
FQuantileDiscrete.split(VRange.of("Age"), 10),
FQuantileDiscrete.split(VRange.of("Fare"), 10),
FQuantileDiscrete.split(VRange.of("SibSp"), 3),
FQuantileDiscrete.split(VRange.of("Parch"), 3),
FQuantileDiscrete.split(VRange.of("FamilySize"), 8),
FRemoveVars.remove(VRange.of("PassengerId,Name"))
};

// print a summary of the transformed data
train.fapply(inputFilters).printSummary();

Frame Summary
* rowCount: 891
* complete: 183/891
* varCount: 12
* varNames: 

 0.   Survived : nom |  4.      SibSp : nom |  8. FamilySize : nom | 
 1.     Pclass : nom |  5.      Parch : nom |  9.     Ticket : nom | 
 2.        Sex : nom |  6.       Fare : nom | 10.      Cabin : nom | 
 3.        Age : nom |  7.   Embarked : nom | 11.      Title : nom | 

* summary: 
 Survived [nom]  Pclass [nom]      Sex [nom]       Age [nom]    SibSp [nom]    Parch [nom]            Fare [nom] 
      0 :   549     3 :   491   male :   577 31.8~36 :    91 -Inf~0 :   608 -Inf~0 :   678    7.854~8.05 :   106 
      1 :   342     1 :   216 female :   314   14~19 :    87  0~Inf :   283  0~Inf :   213     -Inf~7.55 :    92 
                    2 :   184                  41~50 :    78                                   27~39.688 :    91 
                                             -Inf~14 :    77                               39.688~77.958 :    89 
                                             (Other) :   3

We can see that `"Age"` values are now intervals and still $177$ missing values.

The values chosen for quantile numbers is more or less arbitrary. There is no *good* numbers in general, only for some specific purposes.

As promised, we will give a try to another random forest to see if it can better generalize.

In [39]:
RandomSource.setSeed(123);
CForest rf3 = CForest.newModel().runs.set(200);

train = train.fapply(inputFilters);

rf3.fit(train, "Survived");
ClassifierEvaluation.cv(train, "Survived", rf3, 10);

EvalException: null

I tried some ideas to make the forest to generalize better.

* Smaller bootstrap percentage - this could lead to increased independence of trees
* Use `GainRatio` as purity function because sometimes is more conservative
* Use `MinGain` to avoid growing trees to have many leaves with a single instance
* Use `mCols=4`, number of variables used for testing - more than default value, to improve the quality of each tree

These are the results. At a first look might seem like an astonishing result. But we know that the irreducible error for this data set is high and is close to $$0.2$$. It seems obvious that we failed to reduce the variance and we still overfit a lot using this construct. Since this is a tutorial I will not insist on improving this model, but I think that even if it would be improved, the gain would be very small. Perhaps another approach would be better.

## 6. SVM model

SVM (Support Vector Machines) is a nice framework to test new ideas for various types of problems. The power of SMVs comes from their kernels. A kernel is basically a transformation of the original space generated by the input features into another space, often with more dimensions. It's like a feature engineering in a singe function.

But SVMs have a practical problem. The features needs to be numeric and does not allows missing values. This is not a constrain on the algorithm itself. At any moment one can build a kernel for nominal features. But the implemented ones allows only numeric non missing values and is much simpler to shape our data into this format.

How can we do that?

### 6.1 Data preparation

We can use a filter to impute data for missing values. The filter we use is an imputation with a classifierModel of imputation with a regression. The logic is the following: train a classifierModel from a specified set of input features to predict the field with missing values. The data set inside the filter is filtered to contain only instances with non-missing target values.

After we impute the missing values we encode nominal features into numeric features. We can accomplish this task using, again, another filter for this purpose. The name of this filter is `FOneHotEncoding`. What it does is to create a number of numeric feature for level of the nominal variable. Than the values on these numeric variables receives the value of the indicator function. We have $1$ if the level equals the numeric variable's name, $0$ otherwise.

After we have numerical variables, is better to make all the variables to be in the same range. This is not a requirement for SMVs in general. The meaning is to give same weight to all the involved variables. As a side effect it makes the algorithm to run faster. This is due to the fact that the convex optimization problem has smaller chances to have a close-to-flat big surfaces.

Finally, we will remove the not used variables from the frame in order to be prepared for learning.

In [50]:
FFilter[] inputFilters = new FFilter[]{
new CustomFilter(),
FImputeRegression.of(RForest.newRF().runs.set(100),VRange.of("Age,Pclass,Embarked,Sex,Fare,Title"), "Age"),
FImputeClassifier.of(CForest.newModel().runs.set(10),VRange.of("Embarked,Age,Pclass,Sex,Title"), "Embarked"),
FImputeClassifier.of(CForest.newModel().runs.set(100),VRange.of("Age,Pclass,Embarked,Sex,Fare,Ticket"), "Ticket"),
FImputeClassifier.of(CForest.newModel().runs.set(100),VRange.of("Age,Pclass,Embarked,Sex,Fare,Cabin"), "Cabin"),
FOneHotEncoding.on("Sex,Embarked,Title,Cabin"),
FOneHotEncoding.on("Ticket"),
FStandardize.on(VRange.all()),
FRemoveVars.remove(VRange.of("PassengerId,Name,SibSp,Parch"))
};
train.fapply(inputFilters).printSummary();

EvalException: null



```java

```

```
Frame Summary
=============
* rowCount: 891
* complete: 891/891
* varCount: 41
* varNames:

0. Survived : nom | 14. Ticket.3 : num | 28. Cabin.G : num |
1. Pclass : num | 15. Ticket.2 : num | 29. Cabin.D : num |
2. Sex.male : num | 16. Ticket.C : num | 30. Cabin.A : num |
3. Sex.female : num | 17. Ticket.7 : num | 31. Cabin.B : num |
4. Age : num | 18. Ticket.W : num | 32. Cabin.F : num |
5. Fare : num | 19. Ticket.4 : num | 33. Cabin.T : num |
6. Embarked.S : num | 20. Ticket.F : num | 34. Title.Master : num |
7. Embarked.C : num | 21. Ticket.L : num | 35. Title.Rev : num |
8. Embarked.Q : num | 22. Ticket.9 : num | 36. Title.Mr : num |
9. FamilySize : num | 23. Ticket.6 : num | 37. Title.Miss : num |
10. Ticket.A : num | 24. Ticket.5 : num | 38. Title.Dr : num |
11. Ticket.P : num | 25. Ticket.8 : num | 39. Title.Mrs : num |
12. Ticket.S : num | 26. Cabin.C : num | 40. Title.Military : num |
13. Ticket.1 : num | 27. Cabin.E : num |

Survived Pclass Sex.male Sex.female Age
0 : 549 Min. : -1.565 Min. : -1.355 Min. : -0.737 Min. : -2.781
1 : 342 1st Qu. : -0.369 1st Qu. : -1.355 1st Qu. : -0.737 1st Qu. : -0.336
Median : 0.827 Median : 0.737 Median : -0.737 Median : -0.113
Mean : -0.000 Mean : -0.000 Mean : 0.000 Mean : 0.000
2nd Qu. : 0.827 2nd Qu. : 0.737 2nd Qu. : 1.355 2nd Qu. : 0.483
Max. : 0.827 Max. : 0.737 Max. : 1.355 Max. : 2.929
Fare Embarked.S Embarked.C Embarked.Q FamilySize
Min. : -0.648 Min. : -1.632 Min. : -0.482 Min. : -0.303 Min. : -0.561
1st Qu. : -0.489 1st Qu. : -1.632 1st Qu. : -0.482 1st Qu. : -0.303 1st Qu. : -0.561
Median : -0.357 Median : 0.612 Median : -0.482 Median : -0.303 Median : -0.561
Mean : 0.000 Mean : 0.000 Mean : 0.000 Mean : 0.000 Mean : -0.000
2nd Qu. : -0.024 2nd Qu. : 0.612 2nd Qu. : -0.482 2nd Qu. : -0.303 2nd Qu. : 0.059
Max. : 9.662 Max. : 0.612 Max. : 2.073 Max. : 3.297 Max. : 5.637
Ticket.A Ticket.P Ticket.S Ticket.1 Ticket.3
Min. : -0.139 Min. : -0.280 Min. : -0.251 Min. : -0.433 Min. : -0.767
1st Qu. : -0.139 1st Qu. : -0.280 1st Qu. : -0.251 1st Qu. : -0.433 1st Qu. : -0.767
Median : -0.139 Median : -0.280 Median : -0.251 Median : -0.433 Median : -0.767
Mean : -0.000 Mean : -0.000 Mean : -0.000 Mean : -0.000 Mean : 0.000
2nd Qu. : -0.139 2nd Qu. : -0.280 2nd Qu. : -0.251 2nd Qu. : -0.433 2nd Qu. : 1.303
Max. : 7.166 Max. : 3.563 Max. : 3.974 Max. : 2.305 Max. : 1.303
Ticket.2 Ticket.C Ticket.7 Ticket.W Ticket.4
Min. : -0.508 Min. : -0.261 Min. : -0.101 Min. : -0.101 Min. : -0.106
1st Qu. : -0.508 1st Qu. : -0.261 1st Qu. : -0.101 1st Qu. : -0.101 1st Qu. : -0.106
Median : -0.508 Median : -0.261 Median : -0.101 Median : -0.101 Median : -0.106
Mean : 0.000 Mean : -0.000 Mean : -0.000 Mean : 0.000 Mean : 0.000
2nd Qu. : -0.508 2nd Qu. : -0.261 2nd Qu. : -0.101 2nd Qu. : -0.101 2nd Qu. : -0.106
Max. : 1.966 Max. : 3.823 Max. : 9.894 Max. : 9.894 Max. : 9.381
Ticket.F Ticket.L Ticket.9 Ticket.6 Ticket.5
Min. : -0.082 Min. : -0.067 Min. : 0.000 Min. : -0.067 Min. : -0.034
1st Qu. : -0.082 1st Qu. : -0.067 1st Qu. : 0.000 1st Qu. : -0.067 1st Qu. : -0.034
Median : -0.082 Median : -0.067 Median : 0.000 Median : -0.067 Median : -0.034
Mean : -0.000 Mean : -0.000 Mean : 0.000 Mean : -0.000 Mean : -0.000
2nd Qu. : -0.082 2nd Qu. : -0.067 2nd Qu. : 0.000 2nd Qu. : -0.067 2nd Qu. : -0.034
Max. : 12.138 Max. : 14.883 Max. : 0.000 Max. : 14.883 Max. : 29.816
Ticket.8 Cabin.C Cabin.E Cabin.G Cabin.D
Min. : -0.047 Min. : -0.379 Min. : -0.615 Min. : -0.301 Min. : -0.499
1st Qu. : -0.047 1st Qu. : -0.379 1st Qu. : -0.615 1st Qu. : -0.301 1st Qu. : -0.499
Median : -0.047 Median : -0.379 Median : -0.615 Median : -0.301 Median : -0.499
Mean : -0.000 Mean : -0.000 Mean : 0.000 Mean : -0.000 Mean : 0.000
2nd Qu. : -0.047 2nd Qu. : -0.379 2nd Qu. : 1.623 2nd Qu. : -0.301 2nd Qu. : -0.499
Max. : 21.071 Max. : 2.636 Max. : 1.623 Max. : 3.321 Max. : 2.000
Cabin.A Cabin.B Cabin.F Cabin.T Title.Master
Min. : -0.159 Min. : -0.269 Min. : -0.538 Min. : 0.000 Min. : -0.217
1st Qu. : -0.159 1st Qu. : -0.269 1st Qu. : -0.538 1st Qu. : 0.000 1st Qu. : -0.217
Median : -0.159 Median : -0.269 Median : -0.538 Median : 0.000 Median : -0.217
Mean : 0.000 Mean : -0.000 Mean : 0.000 Mean : 0.000 Mean : 0.000
2nd Qu. : -0.159 2nd Qu. : -0.269 2nd Qu. : -0.538 2nd Qu. : 0.000 2nd Qu. : -0.217
Max. : 6.281 Max. : 3.719 Max. : 1.858 Max. : 0.000 Max. : 4.610
Title.Rev Title.Mr Title.Miss Title.Dr Title.Mrs
Min. : -0.082 Min. : -1.183 Min. : -0.510 Min. : -0.089 Min. : -0.409
1st Qu. : -0.082 1st Qu. : -1.183 1st Qu. : -0.510 1st Qu. : -0.089 1st Qu. : -0.409
Median : -0.082 Median : 0.844 Median : -0.510 Median : -0.089 Median : -0.409
Mean : -0.000 Mean : -0.000 Mean : -0.000 Mean : 0.000 Mean : -0.000
2nd Qu. : -0.082 2nd Qu. : 0.844 2nd Qu. : -0.510 2nd Qu. : -0.089 2nd Qu. : -0.409
Max. : 12.138 Max. : 0.844 Max. : 1.959 Max. : 11.231 Max. : 2.440
Title.Military
Min. : -0.082
1st Qu. : -0.082
Median : -0.082
Mean : 0.000
2nd Qu. : -0.082
Max. : 12.138
```

There is a lot of content. Notice that we have numerical variables for each ticket first letter, title, cabin first letter, etc.

## Train a polynomial SVM

A linear kernel is a polynomial kernel with degree 1. We let the $$C$$ parameter to the default value which is $$1$$.

```java
Classifier model = new BinarySMO()
.withInputFilters(inputFilters)
.withC(0.0001)
.withKernel(new PolyKernel(1));
model.train(train, "Survived");
CFit fit = model.fit(test);
new Confusion(train.getVar("Survived"), model.fit(train).firstClasses()).printSummary();
Csv.instance().withQuotes(false).write(SolidFrame.wrapOf(
test.getVar("PassengerId"),
fit.firstClasses().withName("Survived")
), root + "svm1-submit.csv");
cv(train, model);
```
```
> Confusion

Ac\Pr | 0 1 | total
----- | - - | -----
0 | >292 257 | 549
1 | 41 >301 | 342
----- | - - | -----
total | 333 558 | 891


Complete cases 891 from 891
Acc: 0.6655443 (Accuracy )
F1: 0.6621315 (F1 score / F-measure)
MCC: 0.4141426 (Matthew correlation coefficient)
Pre: 0.8768769 (Precision)
Rec: 0.5318761 (Recall)
G: 0.6829274 (G-measure)
Cross validation 10-fold
CV fold: 1, acc: 0.666667, mean: 0.666667, se: NaN
CV fold: 2, acc: 0.764045, mean: 0.715356, se: 0.068857
CV fold: 3, acc: 0.775281, mean: 0.735331, se: 0.059730
CV fold: 4, acc: 0.707865, mean: 0.728464, se: 0.050666
CV fold: 5, acc: 0.719101, mean: 0.726592, se: 0.044077
CV fold: 6, acc: 0.696629, mean: 0.721598, se: 0.041278
CV fold: 7, acc: 0.842697, mean: 0.738898, se: 0.059286
CV fold: 8, acc: 0.730337, mean: 0.737828, se: 0.054972
CV fold: 9, acc: 0.752809, mean: 0.739492, se: 0.051663
CV fold:10, acc: 0.808989, mean: 0.746442, se: 0.053437
=================
mean: 0.746442, se: 0.053437
```

The results are not promising. This is better than random but it is not enough for our purpose. There are some explanations for this result. First one could be that if the space would be linear, than the original feature space would be the same as transformed. This means that a classifierModel as random forest would work well if the linear svm would have worked. This might not be true in general, but in this case it looks like a good explanation. We need to be more flexible.

To increase the flexibility of the model and to allow features to interact with one another we change the degree of the polynomial kernel. This time we will use `degree=3`. Also, we use $$C=0.0001$$ to allow for some errors. This parameter is the factor of the slack regularization constraints of the SVM optimization problem. The bigger the value the more is the penalty for wrong decisions. If the space would be linear separable than one can theoretically set this value as high as possible. But we know it is not. Also we know that we have plenty of irreducible error. As a consequence, it looks like we should decrease the value of this parameter.

```java
Classifier model = new BinarySMO()
.withInputFilters(inputFilters)
.withC(0.0001)
.withKernel(new PolyKernel(1));
model.train(train, "Survived");
CFit fit = model.fit(test);
new Confusion(train.getVar("Survived"), model.fit(train).firstClasses()).printSummary();
Csv.instance().withQuotes(false).write(SolidFrame.wrapOf(
test.getVar("PassengerId"),
fit.firstClasses().withName("Survived")
), root + "svm1-submit.csv");
cv(train, model);
```
```
> Confusion

Ac\Pr | 0 1 | total
----- | - - | -----
0 | >472 77 | 549
1 | 48 >294 | 342
----- | - - | -----
total | 520 371 | 891


Complete cases 891 from 891
Acc: 0.8597082 (Accuracy )
F1: 0.8830683 (F1 score / F-measure)
MCC: 0.7097044 (Matthew correlation coefficient)
Pre: 0.9076923 (Precision)
Rec: 0.859745 (Recall)
G: 0.8833934 (G-measure)
Cross validation 10-fold
CV fold: 1, acc: 0.822222, mean: 0.822222, se: NaN
CV fold: 2, acc: 0.786517, mean: 0.804370, se: 0.025248
CV fold: 3, acc: 0.853933, mean: 0.820891, se: 0.033728
CV fold: 4, acc: 0.808989, mean: 0.817915, se: 0.028174
CV fold: 5, acc: 0.797753, mean: 0.813883, se: 0.026012
CV fold: 6, acc: 0.786517, mean: 0.809322, se: 0.025809
CV fold: 7, acc: 0.831461, mean: 0.812484, se: 0.025002
CV fold: 8, acc: 0.876404, mean: 0.820474, se: 0.032350
CV fold: 9, acc: 0.764045, mean: 0.814204, se: 0.035631
CV fold:10, acc: 0.820225, mean: 0.814806, se: 0.033647
=================
mean: 0.814806, se: 0.033647
```

This time the results are promising. We achieved a training error which is not close to zero and the cross validation errors are close to our desired results. We definitely should try this classifierModel.

![SVM1](images/titanic-svm1-submit.png)

We have a better score also on public leader board. Which is very fine. Usually in this competition a score in $$0.75-0.78$$ is fine and one in $$0.78-0.81$$ is excellent.

## Tuning manually the SVM

We can work more on SVMs. One thing which deserves a try is the radial basis kernel. This is similar with working in an infinite dimensional space! We tried some `RBFKernel` approaches, but much better results gave the `CauchyKernel`. The `CauchyKernel` works in a similar way like a RBF kernel. The difference which sometimes is important is that it is a distribution with tails fatter than Gaussian distribution. This produces an effect of long distance influence. This is reasonable to use in this problem because we know we have noise. We can think that a kernel which acts on wider ranges is better if it is combined with a small value for $$C$$.

After some manual tuning we arrived at the following classifierModel.

```java
Classifier model = new BinarySMO()
.withInputFilters(inputFilters)
.withC(1)
.withTol(1e-10)
.withKernel(new CauchyKernel(25));
```
```
> Confusion

Ac\Pr | 0 1 | total
----- | - - | -----
0 | >520 29 | 549
1 | 104 >238 | 342
----- | - - | -----
total | 624 267 | 891


Complete cases 891 from 891
Acc: 0.8507295 (Accuracy )
F1: 0.8866155 (F1 score / F-measure)
MCC: 0.6826819 (Matthew correlation coefficient)
Pre: 0.8333333 (Precision)
Rec: 0.9471767 (Recall)
G: 0.8884334 (G-measure)
Cross validation 10-fold
CV fold: 1, acc: 0.811111, mean: 0.811111, se: NaN
CV fold: 2, acc: 0.786517, mean: 0.798814, se: 0.017391
CV fold: 3, acc: 0.865169, mean: 0.820932, se: 0.040235
CV fold: 4, acc: 0.797753, mean: 0.815137, se: 0.034836
CV fold: 5, acc: 0.831461, mean: 0.818402, se: 0.031040
CV fold: 6, acc: 0.842697, mean: 0.822451, se: 0.029481
CV fold: 7, acc: 0.820225, mean: 0.822133, se: 0.026926
CV fold: 8, acc: 0.820225, mean: 0.821895, se: 0.024937
CV fold: 9, acc: 0.797753, mean: 0.819212, se: 0.024676
CV fold:10, acc: 0.831461, mean: 0.820437, se: 0.023585
=================
mean: 0.820437, se: 0.023585
```

This classifierModel has similar results, but there are reasons to believe that it is slightly better than previous. The training error smaller. But we know that training error is not a good estimator. The 10 fold cv is greater. This is a good sign. A better interpretation would be that the gap between those two has shrunken and this is a good thing. A new submit on kaggle follows.

![SVM2](images/titanic-svm2-submit.png)

Well we are really, really close to our psychological milestone of $$0.8$$. Perhaps some tuning will give more results. This is true in general. However, next section provides you with a better approach which usually provides some gain in accuracy: stacking.

## 7. Stacking classifiers

Using random forests or SVMs did not provided us with a result over $$0.8$$. We have two very different types of models which performed well. For the sole purpose of prediction we use a nice ensemble technique which often provides good prediction performance gain. This technique is called stacking.

The idea behind stacking is that one can explore the space of the solutions with different approaches. Each approach (or statistical model) is basically an interpretation of the solution. But often times a proper interpretation is really hard to find. Each interpretation of the solution can have good points and weak points. The idea is to blend those interpretations into a single one in a way that we try somehow to keep what is string from each individual classifierModel.

A stacking classifierModel take some base learners and train them on training data. The results of the base learners are used as input for a stacking learner. This stacking learner is trained on the output of base learners and target variable and is finally used for prediction.

```java
Classifier model = new CStacking()
.withLearners(
new BinarySMO()
.withInputFilters(inputFilters)
.withC(1)
.withTol(1e-10)
.withKernel(new CauchyKernel(20)),
CForest.newRF()
.withInputFilters(inputFilters)
.withMCols(4)
.withBootstrap(0.07)
.withClassifier(CTree.newCART()
.withFunction(CTreePurityFunction.GainRatio)
.withMinGain(0.001))
.withRuns(200)
).withStacker(CForest.newRF().withBootstrap(0.3)
.withRuns(200)
.withClassifier(CTree.newCART()
.withFunction(CTreePurityFunction.GainRatio)
.withMinGain(0.05)));
```

Usually one uses a binary logistic regression model but it provided weak results. What looked much better is another random forrest classifierModel. However the stacking model uses a big value for minimum gain parameter because we want to act as an draft average over the results.

```
> Confusion

Ac\Pr | 0 1 | total
----- | - - | -----
0 | >522 27 | 549
1 | 87 >255 | 342
----- | - - | -----
total | 609 282 | 891


Complete cases 891 from 891
Acc: 0.8720539 (Accuracy )
F1: 0.9015544 (F1 score / F-measure)
MCC: 0.7281918 (Matthew correlation coefficient)
Pre: 0.8571429 (Precision)
Rec: 0.9508197 (Recall)
G: 0.902767 (G-measure)
Cross validation 10-fold
CV fold: 1, acc: 0.888889, mean: 0.888889, se: NaN
CV fold: 2, acc: 0.786517, mean: 0.837703, se: 0.072388
CV fold: 3, acc: 0.842697, mean: 0.839367, se: 0.051267
CV fold: 4, acc: 0.820225, mean: 0.834582, se: 0.042940
CV fold: 5, acc: 0.808989, mean: 0.829463, se: 0.038908
CV fold: 6, acc: 0.853933, mean: 0.833541, se: 0.036206
CV fold: 7, acc: 0.786517, mean: 0.826824, se: 0.037527
CV fold: 8, acc: 0.898876, mean: 0.835830, se: 0.043082
CV fold: 9, acc: 0.831461, mean: 0.835345, se: 0.040326
CV fold:10, acc: 0.775281, mean: 0.829338, se: 0.042500
=================
mean: 0.829338, se: 0.042500
```
Again, the results are promising. The space between training error and cross validation error is smaller and our expectations grows.

![Stacking with a random forest](images/titanic-stacking1-submit.png)

Finally our target performance was achieved!!

**Note:**

The general advice in real life is to not fight for each piece of performance measure. It really depends on the question one wants to answer. Often measures like ROC or partial ROC are much better than error frequency. We fixed this milestone because we know it is possible and because it looks like a psychological difficulty. (that $$0.799$$ is outrageous).